In [ ]:
install.packages("devtools")
install.packages("dplyr")
install.packages("rmarkdown")
install.packages("ggplot2")
install.packages("broom")
install.packages("gridExtra")
install.packages("shiny")
install.packages("cubature")
install.packages("tidyr")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [3]:
library(ggplot2)
library(dplyr)
library(tidyverse)
library(reshape2)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths



In [4]:
load("../data/brfss2013")
df <- brfss2013

In [5]:
nrow(brfss2013)

[1] 491775

### Data

The BRFSS data is a random sample of adults 18 years or older from all US states and outlying territories. The survey questions are asked on a landline phone or cell phone from residentials household where they select a random adult or from college housing. Cell phones are also used for contact purposes. The survey could potentially be a stratified sample (groups separated by college and non college housing) but it doesn't appear they this is the case. This is not assignment as no procedures are being performed to assign participants to control or experimental groups.

### Research Question (RQ) 1
    a) Does household income correlate with a decision to not seek medical attention when needed?
    b) Is there a decision difference between the sexes?
    
### Reason for Interest
    I don't think I've seen a statistical analysis addressing this specific correlation, though I'm 
    sure it has been done. I'm curious to see if there is a decision difference amongst the sexes, 
    about medical care, regarding their financial situation.

### Exploratory Data Analysis (EDA) for RQ1

In [6]:
med <- (
    
    df %>%  
    select(medcost, income2, sex) %>%
    drop_na()
    
)

In [7]:
head(med)

medcost,income2,sex
No,"Less than $75,000",Female
No,"$75,000 or more",Female
No,"$75,000 or more",Female
No,"Less than $75,000",Female
No,"Less than $50,000",Male
No,"$75,000 or more",Female


In [ ]:
ggplot(med, aes(x=income2, fill=medcost))+ geom_bar(position="fill") + coord_flip() + facet_wrap(~sex)

### Summary Statistics for RQ1

In [7]:
library(data.table)


Attaching package: ‘data.table’

The following objects are masked from ‘package:reshape2’:

    dcast, melt

The following object is masked from ‘package:purrr’:

    transpose

The following objects are masked from ‘package:dplyr’:

    between, first, last



In [8]:
new_df <- 
(
    
    med %>%  
    group_by(medcost, income2, sex) %>% 
    tally() %>%
    spread(medcost, n) %>%
    unite("income_by_sex", income2:sex) %>%
    mutate(total=rowSums(select_if(., is.numeric)))
)


In [9]:
pcts <- new_df %>% select(2:3)
pcts <- pcts/rowSums(pcts)
names(pcts) <- c("pct_cost_issue", "pct_cost_not_issue")
final_df <- cbind(new_df, pcts)
setnames(final_df, old=c("Yes","No"), new=c("count_cost_issue", "count_cost_not"))
summary_df <-final_df[order(-final_df$pct_cost_issue),]
summary_df

,income_by_sex,count_cost_issue,count_cost_not,total,pct_cost_issue,pct_cost_not_issue
1,"Less than $10,000_Male",2400,5862,8262,0.29048656,0.7095134
2,"Less than $10,000_Female",4888,12172,17060,0.28651817,0.7134818
4,"Less than $15,000_Female",4513,12998,17511,0.25772372,0.7422763
3,"Less than $15,000_Male",2320,6847,9167,0.25308171,0.7469183
6,"Less than $20,000_Female",5177,17066,22243,0.23274738,0.7672526
5,"Less than $20,000_Male",2896,9613,12509,0.23151331,0.7684867
8,"Less than $25,000_Female",5339,20607,25946,0.20577353,0.7942265
7,"Less than $25,000_Male",2945,12731,15676,0.18786680,0.8121332
10,"Less than $35,000_Female",4608,24575,29183,0.15790015,0.8420999
9,"Less than $35,000_Male",2535,17045,19580,0.12946885,0.8705312


### Narrative RQ1

To address the research question, I grouped filtered the BRFSS2013 data by medcost, sex and income (specifically the income2 column). I grouped all the data by the column headers and counted the totals of each grouping. Percentages were taken upon whether a group didn't seek medical attention when needed. The figure reflects the information in the summary statistics where regardless of sex, if individuals in that group make less than $10k more of them are likely to have responded yes, they DID NOT seek medical attention when needed. One could deduce from this information that is likely due to the high cost of medical care. A larger percentage of income would have to be spend on getting care.